# CIA Triad and Cryptographic Methods

## Contents
1. CIA Triad Definition
2. Confidentiality
3. Integrity
4. Availability
5. Practical Examples

## Prerequisites:

```bash
pip install cryptography hmac
```

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend
from cryptography.fernet import Fernet
import hashlib
import time
import hmac as hmac_lib

## 1. CIA Triad Definition

**CIA Triad** = Three fundamental principles of information security

### Components:

| Principle | Definition | Goal |
|-----------|------------|------|
| **Confidentiality** | Data accessible only to authorized parties | Prevent unauthorized disclosure |
| **Integrity** | Data remains accurate and unmodified | Prevent unauthorized modification |
| **Availability** | Data accessible when needed | Ensure system uptime and access |

### Security Balance:
- All three must be balanced for effective security
- Strengthening one may affect others
- Different scenarios prioritize different aspects

## 2. Confidentiality

**Definition:** Ensuring information is disclosed only to authorized parties

### Cryptographic Methods:
- **Encryption** (symmetric/asymmetric)
- **Access controls** with authentication
- **Key management**
- **Secure channels** (TLS/SSL)

### How Cryptography Helps:
- Encrypts data at rest and in transit
- Only authorized parties with keys can decrypt
- Protects against eavesdropping and theft

### Example 1: File Encryption (AES)

In [33]:
# Sensitive data
confidential_data = "Patient medical records: Marcel T.O, Diagnosis: Confidential"

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)

# Encrypt
encrypted = cipher.encrypt(confidential_data.encode())

print(f"Original: {confidential_data}")
print(f"\nEncrypted: {encrypted[:60]}...")

# Decrypt (authorized access)
decrypted = cipher.decrypt(encrypted).decode()
print(f"\nWith key: {decrypted}")
print("\nConfidentiality maintained through encryption")

Original: Patient medical records: Marcel T.O, Diagnosis: Confidential

Encrypted: b'gAAAAABpIbsl2gJzucupKYakIPUtNeKfZXdq7FfmXHMiR8vFFxemotiekbxS'...

With key: Patient medical records: Marcel T.O, Diagnosis: Confidential

Confidentiality maintained through encryption


### Example 2: HTTPS/TLS Communication

In [34]:
# Simulate asymmetric encryption for key exchange
private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=2048, backend=default_backend()
)
public_key = private_key.public_key()

# Sensitive data (e.g., credit card)
credit_card = "4532-1234-5678-9010"

# Encrypt with public key
encrypted = public_key.encrypt(
    credit_card.encode(),
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
    ),
)

print(f"Credit card: {credit_card}")
print(f"Encrypted (first 60 bytes): {encrypted[:60].hex()}...")
print("\nData encrypted in transit (HTTPS)")
print("Eavesdroppers cannot read the data")
print("Only server with private key can decrypt")

Credit card: 4532-1234-5678-9010
Encrypted (first 60 bytes): 0f4f54d9320ca7c5b5fcecf48caa86ba84c3d16090d21835c47fa9d7c707641edfc9826a9b623f0b4219ebf6ec526477d9661550bcb8a6756bc5283d...

Data encrypted in transit (HTTPS)
Eavesdroppers cannot read the data
Only server with private key can decrypt


### Example 3: Database Encryption

In [35]:
# Simulate database records
key = Fernet.generate_key()
cipher = Fernet(key)

users = [
    {"id": 1, "name": "Marcel", "ssn": "123-45-6789"},
    {"id": 2, "name": "Rick Grimes", "ssn": "987-65-4321"},
]

# Encrypt sensitive fields
for user in users:
    user["ssn_encrypted"] = cipher.encrypt(user["ssn"].encode())
    del user["ssn"]  # Remove plaintext

print("Encrypted Database Records:")
for user in users:
    print(f"  ID: {user['id']}, Name: {user['name']}")
    print(f"  SSN: {user['ssn_encrypted'][:50]}...")

print("\nSSN encrypted at rest")
print("Database breach doesn't expose sensitive data")
print("Encryption key stored separately (HSM/Key vault)")

Encrypted Database Records:
  ID: 1, Name: Marcel
  SSN: b'gAAAAABpIbslVa9JTN6f7YTf7uAgsvMrx7SNBHWrrjgZvIK16G'...
  ID: 2, Name: Rick Grimes
  SSN: b'gAAAAABpIbslY63u0vFPCXj0aAL8ezb-nN4cKdh7LYVAFbzjKi'...

SSN encrypted at rest
Database breach doesn't expose sensitive data
Encryption key stored separately (HSM/Key vault)


## 3. Integrity

**Definition:** Ensuring information remains accurate and unmodified

### Cryptographic Methods:
- **Hash functions** (SHA-256, SHA-3)
- **Digital signatures** (RSA, ECDSA)
- **HMAC** (Hash-based Message Authentication Code)
- **Checksums** and MACs

### How Cryptography Helps:
- Detects any modification to data
- Verifies data authenticity
- Prevents tampering
- One-way functions make reversal impossible

### Example 1: File Integrity Check (Hash)

In [36]:
# Original file content
def compute_sha256(file_path):
    """Compute the SHA-256 hash of a file."""
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        # Read and update hash string value in blocks of 4K
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

file_path = "./cia_triade.ipynb"
hash_value = compute_sha256(file_path)
# Published hash (for demonstration, we use the computed one)
public_hash = hash_value

print(f"Published SHA-256: {public_hash}")

print(f"\nCalculated SHA-256: {hash_value}")
print(f"\nMatch: {hash_value == public_hash}")
print("File not corrupted or tampered with")

# Simulate tampering
tampered_file = b"Ubuntu-22.04-desktop-amd64.iso (MODIFIED MALWARE)"
digest3 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest3.update(tampered_file)
tampered_hash = digest3.finalize()

print(f"\nTampered SHA-256: {tampered_hash.hex()}")
print(f"Match: {hash_value == tampered_hash.hex()}")
print("✗ Integrity check failed - file modified!")

Published SHA-256: 306247148824174039d5e2d6ac721a565616e9d05c8debd3e3e768223735257c

Calculated SHA-256: 306247148824174039d5e2d6ac721a565616e9d05c8debd3e3e768223735257c

Match: True
File not corrupted or tampered with

Tampered SHA-256: de8b39be3b5ab2c15ddddbcc29ada2a362225ae8de9f070afbe80c23aeefde76
Match: False
✗ Integrity check failed - file modified!


### Example 2: Digital Signatures

In [37]:
# Developer's key pair
private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=2048, backend=default_backend()
)
public_key = private_key.public_key()

# Software code
software_code = b"Application.exe binary data"

# Developer signs the code
signature = private_key.sign(
    software_code,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)

print("Software: Application.exe")
print(f"Signature (first 40 bytes): {signature[:40].hex()}...")

# User verifies signature
try:
    public_key.verify(
        signature,
        software_code,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256(),
    )
    print("\nSignature valid")
    print("Code verified from trusted developer")
    print("Code not modified since signing")
except Exception:
    print("\nSignature invalid - code tampered!")

# Test with modified code
modified_code = b"Application.exe binary data MALWARE"
try:
    public_key.verify(
        signature,
        modified_code,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256(),
    )
    print("\nSignature valid")
except Exception:
    print("\nModified code: Signature verification failed!")
    print("Integrity compromised")

Software: Application.exe
Signature (first 40 bytes): 1032e25e07dfa3666588d23de523c2a9fe564dba3f93767efb8291436085b62c770539648d2f10f7...

Signature valid
Code verified from trusted developer
Code not modified since signing

Modified code: Signature verification failed!
Integrity compromised


## 4. Availability

**Definition:** Ensuring authorized users can access information when needed

### Cryptographic Methods:
- **Key escrow/backup** systems
- **Redundancy** in key management
- **Access control** mechanisms
- **Protection against DoS** attacks

### How Cryptography Helps:
- Prevents denial-of-service attacks
- Ensures authorized access to encrypted data
- Key recovery mechanisms
- Load balancing with cryptographic authentication

### Note:
Cryptography indirectly supports availability but primarily focuses on confidentiality and integrity. Availability often requires additional infrastructure (redundancy, backups, DDoS protection).

### Example 1: Key Backup

In [38]:
# Master encryption key
master_key = Fernet.generate_key()
print(f"Master key: {master_key.decode()}")

# Encrypt sensitive data
cipher = Fernet(master_key)
data = b"Critical business data"
encrypted_data = cipher.encrypt(data)

print(f"\nEncrypted data: {encrypted_data[:50]}...")

# Key escrow: Split key across multiple secure locations
key_parts = [master_key[:22], master_key[22:]]

print(f"\nKey split into {len(key_parts)} parts:")
print(f"  Part 1 (HSM-1): {key_parts[0]}")
print(f"  Part 2 (HSM-2): {key_parts[1]}")

# Simulate key recovery
print("\nScenario: Primary key lost, recovering from backup...")
recovered_key = key_parts[0] + key_parts[1]

cipher2 = Fernet(recovered_key)
recovered_data = cipher2.decrypt(encrypted_data)

print("Key recovered from backup")
print(f"Data decrypted: {recovered_data.decode()}")
print("\nAvailability maintained through key backup")
print("No data loss even if primary key lost")

Master key: BRJ-8JMhHFJJ20aDdGMu1-M8Z5ZAENqaoiVDmlJVQtE=

Encrypted data: b'gAAAAABpIbsldn5BeBwhI9sSu6bnDCt7Vsua4HFfnJM5MPH1LB'...

Key split into 2 parts:
  Part 1 (HSM-1): b'BRJ-8JMhHFJJ20aDdGMu1-'
  Part 2 (HSM-2): b'M8Z5ZAENqaoiVDmlJVQtE='

Scenario: Primary key lost, recovering from backup...
Key recovered from backup
Data decrypted: Critical business data

Availability maintained through key backup
No data loss even if primary key lost


### Example 2: Rate Limiting with Cryptographic Tokens

In [39]:
# Server secret for token generation
server_secret = b"secret_key_for_rate_limiting"


def generate_token(user_id, timestamp):
    """Generate cryptographic rate limit token"""
    message = f"{user_id}:{timestamp}".encode()
    token = hmac_lib.new(server_secret, message, hashlib.sha256).hexdigest()
    return token


def verify_token(user_id, timestamp, token):
    """Verify rate limit token"""
    expected = generate_token(user_id, timestamp)
    return hmac_lib.compare_digest(token, expected)


# Legitimate user
user_id = "alice"
timestamp = int(time.time())
token = generate_token(user_id, timestamp)

print(f"User: {user_id}")
print(f"Token: {token[:40]}...")

if verify_token(user_id, timestamp, token):
    print("\nToken valid - Request allowed")
    print("Legitimate users can access service")

# Attacker tries to forge token
fake_token = "forged_token_12345" + "0" * 24
if not verify_token(user_id, timestamp, fake_token):
    print("\nInvalid token - Request blocked")
    print("DDoS/abuse attempts rejected")
    print("Service availability protected")

User: alice
Token: bffef24716f63b8bcba98146acd354dea3ffaf7d...

Token valid - Request allowed
Legitimate users can access service

Invalid token - Request blocked
DDoS/abuse attempts rejected
Service availability protected


## Summary

### CIA Triad Overview:

| Principle | Cryptographic Methods | Practical Examples |
|-----------|----------------------|--------------------|
| **Confidentiality** | • Symmetric encryption (AES)<br>• Asymmetric encryption (RSA)<br>• TLS/SSL | 1. File/disk encryption<br>2. HTTPS for web traffic<br>3. Database column encryption |
| **Integrity** | • Hash functions (SHA-256)<br>• Digital signatures (RSA)<br>• HMAC | 1. File checksum verification<br>2. Code signing<br>3. API message authentication |
| **Availability** | • Key backup/escrow<br>• Rate limiting tokens<br>• Multi-factor authentication | 1. Key recovery systems<br>2. DDoS protection<br>3. Backup authentication codes |

### How Cryptography Achieves CIA:

**Confidentiality:**
- Encryption makes data unreadable without key
- Only authorized parties with keys can access
- Protects data at rest and in transit

**Integrity:**
- Hash functions detect any modification
- Digital signatures prove authenticity
- One-way functions prevent forgery

**Availability:**
- Key backup prevents data loss
- Cryptographic tokens prevent abuse
- MFA provides alternative access methods

### Real-World Application:
Most secure systems use **all three principles together**:
- Online banking: Encryption (C) + signatures (I) + MFA (A)
- Cloud storage: Encryption (C) + checksums (I) + key recovery (A)
- Email (PGP/S/MIME): Encryption (C) + signing (I) + key backup (A)

### Best Practices:
- Don't prioritize one principle at expense of others
- Use established cryptographic libraries
- Implement defense in depth
- Regular security audits
- Key management is critical for all three